# 教材をリリースする
---

作成した教材をリリースします。リリースした教材は、フォルダ「textbook」で保持されます。<br/>

次のセルを実行して下さい。
* 'MATELIALS' には、リリースする教材のファイル名、フォルダ名を列挙します。
```
MATELIALS="notebook-1.ipynb additional/"
```

In [ ]:
#!/bin/bash

MATELIALS=""

ROOT_DIR=/jupyter
SRC_TEXTBOOK=$ROOT_DIR/admin/$USER
DST_TEXTBOOK=$ROOT_DIR/admin/textbook
EXCLUDE_OPTIONS="--exclude=.* --exclude=*"

# add include options for sub-directory
function add_dir_include_options () {
  (
  dir_name=$1
  path=$2  
  add_options="--include=$dir_name/* "
  current_dir="$path/$dir_name"
  for material in $(ls "$current_dir"); do
    [ -d "$current_dir/$material" ] && {
      add_options=${add_options}$(add_dir_include_options "$dir_name/$material" "$path")
    }
  done
  echo $add_options" "
  )
}

# build include options
include_options=""
for material in $MATELIALS; do
  material=${material%/}  
  include_options="${include_options}--include=$material "
  [ -d "$SRC_TEXTBOOK/$material" ] && {
    include_options=${include_options}$(add_dir_include_options "$material" "$SRC_TEXTBOOK")
  }
done

# release materials
rsync -rltuvO --delete $include_options $EXCLUDE_OPTIONS $SRC_TEXTBOOK/ $DST_TEXTBOOK

# deleting the release canceled materials
echo
echo "deleting the release canceled materials"
for dst in $(ls -a "$DST_TEXTBOOK"); do
  [[ "$dst" = '.' || "$dst" = '..' ]] && {
    continue
  }
  found=0
  for material in $MATELIALS; do
    material=${material%/}  
    if [ "$material" = "$dst" ]; then
      found=1
      break
    fi
  done
  if [ "$found" -eq 0 ]; then
    rm  -rf $DST_TEXTBOOK/$dst
    echo "'$dst' was deleted.'"
  fi
done